In [ ]:
http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

!unzip "/content/drive/MyDrive/Colab Notebooks/Word2Vec/cbow_s300.zip"

import pandas as pd
artigo_treino = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/treino.csv")
artigo_teste = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/teste.csv")

from sklearn.feature_extraction.text import CountVectorizer

texto = [
    "Este produto é muito bom",
    "Este produto é muito ruim"
]

vetorizador = CountVectorizer()
vetorizador.fit(texto)
print(vetorizador.vocabulary_)

from gensim.models import KeyedVectors
modelo = KeyedVectors.load_word2vec_format("cbow_s300.txt")

import nltk
import string
nltk.download('punkt')

def tokenizador(texto):
  texto = texto.lower()
  lista_alfanumerico = []

  for token_valido in nltk.word_tokenize(texto):
    if token_valido in string.punctuation: continue
    lista_alfanumerico.append(token_valido)
  
  return lista_alfanumerico

import nltk
import string
nltk.download('punkt')

def tokenizador(texto):
  texto = texto.lower()
  lista_alfanumerico = []

  for token_valido in nltk.word_tokenize(texto):
    if token_valido in string.punctuation: continue
    lista_alfanumerico.append(token_valido)
  
  return lista_alfanumerico

def matriz_vetores(textos):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras_numeros)

  return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

LR = LogisticRegression(max_iter = 200)
LR.fit(matriz_vetores_treino, artigo_treino.category)
label_prevista = LR.predict(matriz_vetores_teste)
CR = classification_report(artigo_teste.category, label_prevista)
print(CR)

from sklearn.dummy import DummyClassifier

DC = DummyClassifier()
DC.fit(matriz_vetores_treino, artigo_treino.category)
label_previsao_dc = DC.predict(matriz_vetores_teste)
CR_dummy = classification_report(artigo_teste.category, label_previsao_dc)
print(CR_dummy)

modelo_skipgram = KeyedVectors.load_word2vec_format("skip_s300.txt")

def combinacao_de_vetores_por_soma_skipgram(palavras_numeros):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try:
      vetor_resultante =+ modelo_skipgram.get_vector(pn)
    except KeyError:
      if pn.isnumeric():
        pn = "0"*len(pn)
        vetor_resultante =+ modelo_skipgram.get_vector(pn)
      else:
        vetor_resultante =+ modelo_skipgram.get_vector('unknown')
  return vetor_resultante

def matriz_vetores_skipgram(textos):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras_numeros = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma_skipgram(palavras_numeros)

matriz_vetores_treino_skipgram = matriz_vetores_skipgram(artigo_treino.title)
matriz_vetores_teste_skipgram = matriz_vetores_skipgram(artigo_teste.title)

LR_skipgram = LogisticRegression(max_iter = 200)
LR_skipgram.fit(matriz_vetores_treino_skipgram, artigo_treino.category)
label_previsao_skipgram = LR_skipgram.predict(matriz_vetores_teste_skipgram)
CR_skipgram = classification_report(artigo_teste.category, label_previsao_skipgram)
print(CR_skipgram)